In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print('GPU enable')
    except Exception as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU enable


### Analog ```tf.keras.utils.image_dataset_from_directory```

In [3]:
path = './archive/'
train_path = f'{path}train/'
test_path = f'{path}test/'
valid_path = f'{path}valid/'

In [86]:
class_to_id = {}

def pd_from_path(directory: str) -> pd.DataFrame:
    global class_to_id
    data = []
    for path, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.gif')):
                class_name = os.path.basename(path)
                if class_name not in class_to_id:
                    class_to_id[class_name] = len(class_to_id)
                data.append((f'{path}/{file}', class_to_id[class_name]))

    return pd.DataFrame(data, columns=['imagepath', 'label'])

In [ ]:
deleted = []
for file in pd_from_path(path).imagepath:
    try:
      img_bytes = tf.io.read_file(file)
      decoded_img = tf.io.decode_image(img_bytes)
    except Exception:
      deleted.append(file)
      os.remove(file)

In [87]:
def tf_load_image(image_path: str, crop_size: tuple[int, int] = (256, 256)):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image, expand_animations=False, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, size=[*crop_size])
    return image

In [88]:
def extract_and_zip(df: pd.DataFrame, crop_size: tuple[int, int] = (256, 256), batch_size: int = 32):
    image_dataset = tf.data.Dataset.from_tensor_slices(df.imagepath)
    label_dataset = tf.data.Dataset.from_tensor_slices(df.label)
    dataset = tf.data.Dataset.zip((image_dataset.map(lambda x: tf_load_image(x, crop_size)), label_dataset))
    return dataset.batch(batch_size)

In [89]:
def load_dataset_from_directory(
        directory: str = './',
        batch_size: int = 32,
        image_size: tuple[int, int] = (256, 256),
        shuffle: bool = True,
        seed: int = None,
        validation_split: float = None,
        subset: str = None
) -> tf.data.Dataset:
    
    df = pd_from_path(directory)
    
    if shuffle:
        df = df.sample(frac=1, random_state=seed)
    
    if validation_split:
        index = round(len(df) * validation_split)
        train_dataset = df[index:]
        valid_dataset = df[:index]

        if subset == 'training':
            df = train_dataset
        elif subset == 'validation':
            df = valid_dataset
        elif subset == 'both':
            return extract_and_zip(train_dataset), extract_and_zip(valid_dataset)
    
    
    return extract_and_zip(df, image_size, batch_size)
        

In [90]:
dataset = load_dataset_from_directory(path)

In [100]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(256, 256, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(525))

In [101]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy', optimizer='adam')

In [102]:
model.fit(dataset, epochs=1)

2648/2809 [===========================>..] - ETA: 17s - loss: 5.0093 - accuracy: 0.1327

### Function

In [70]:
def load_dataset(path: str, batch_size: int, image_size: tuple[int, int], shuffle: bool, split: str) -> tuple[tf.data.Dataset, dict[int, str]]:
    '''Given a `path` to a csv index file loads one of the dataset splits. Paths in the index are assumed to be relative to the csv file. The file contains three columns: "filepaths", "labels" and "data set", path to the image, image label and dataset split respectively.

    Arguments:
        path: path to the csv index file
        batch_size: size of batches in the dataset
        image_size: size to resize the images to
        shuffle: whether to shuffle the index. If False original index order is preserved
        split: split to use. One of "train", "valid" or "test"

    Returns:
        The loaded dataset
        A dictionary mapping class indices to class names'''
    data = pd.read_csv(path)
    classes = pd.unique(data.labels)
    class_to_id = {name: i for i, name in enumerate(classes)}
    data.insert(3, 'labels_id', [class_to_id[name] for name in data.labels], True)

    if shuffle:
        data = data.sample(frac=1)
    
    if split:
        data = data[data['data set'] == split]

    image_dataset = tf.data.Dataset.from_tensor_slices(data.filepaths)
    label_dataset = tf.data.Dataset.from_tensor_slices(data.labels_id)
    
    dataset = tf.data.Dataset.zip((image_dataset.map(lambda x: tf_load_image(x, image_size)), label_dataset))

    return dataset.batch(batch_size), {i: name for i, name in enumerate(classes)}

In [ ]:
load_dataset('./archive/new_birds.csv', 32, (256, 256), False, None)

(<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 {0: 'ABBOTTS BABBLER',
  1: 'ABBOTTS BOOBY',
  2: 'ABYSSINIAN GROUND HORNBILL',
  3: 'AFRICAN CROWNED CRANE',
  4: 'AFRICAN EMERALD CUCKOO',
  5: 'AFRICAN FIREFINCH',
  6: 'AFRICAN OYSTER CATCHER',
  7: 'AFRICAN PIED HORNBILL',
  8: 'AFRICAN PYGMY GOOSE',
  9: 'ALBATROSS',
  10: 'ALBERTS TOWHEE',
  11: 'ALEXANDRINE PARAKEET',
  12: 'ALPINE CHOUGH',
  13: 'ALTAMIRA YELLOWTHROAT',
  14: 'AMERICAN AVOCET',
  15: 'AMERICAN BITTERN',
  16: 'AMERICAN COOT',
  17: 'AMERICAN FLAMINGO',
  18: 'AMERICAN GOLDFINCH',
  19: 'AMERICAN KESTREL',
  20: 'AMERICAN PIPIT',
  21: 'AMERICAN REDSTART',
  22: 'AMERICAN ROBIN',
  23: 'AMERICAN WIGEON',
  24: 'AMETHYST WOODSTAR',
  25: 'ANDEAN GOOSE',
  26: 'ANDEAN LAPWING',
  27: 'ANDEAN SISKIN',
  28: 'ANHINGA',
  29: 'ANIANIAU',
  30: 'ANNAS HUMMINGBIRD',
  31: 'ANTBIRD',
  32: 'ANTILLEAN EUPHONIA',
  33: 

In [75]:
dataset2, cindex = load_dataset('./archive/new_birds.csv', 32, (256, 256), False, None)

In [76]:
inputs2 = tf.keras.layers.Input((256, 256, 3), name='input', dtype=tf.float32)
x2 = tf.keras.layers.Dense(256, name='pre-prediction')(inputs2)
x2 = tf.keras.layers.Dense(1, name='prediction')(x2)
model2 = tf.keras.Model(inputs=inputs2, outputs=x2, name='craft')

In [77]:
model2.compile(loss='sparse_categorical_crossentropy', metrics='accuracy', optimizer='adam')

In [78]:
model2.fit(dataset2, epochs=1)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd1 in position 165: invalid continuation byte

### Interface ```tf.keras.utils.Sequence```